In [15]:
import json
import signal
import sys
import datetime
import math
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
from confluent_kafka import Producer, Consumer, KafkaException, KafkaError

In [2]:
FEATURE_COLS = [
    "EAR",
    "MAR",
    "yawn_rate_per_min",
    "blink_rate_per_min",
    "avg_blink_dur_sec",
    "longest_eye_closure_sec",
]

In [ ]:
kafka_conf = {
    "bootstrap.servers": "kafka.dongango.com:9094",
}

In [4]:
conf = {
        **kafka_conf,
        "group.id": "zolgima-test-consumer-group",
        "enable.auto.commit": False,  # 수동 커밋
        "auto.offset.reset": "earliest",  # 처음부터 읽고 싶을 때
        "session.timeout.ms": 45000,
      }

c = Consumer(conf)
md = c.list_topics(timeout=5.)

In [5]:
mp4_topics = []
for tname, tmeta in md.topics.items():
    if tname.endswith(".mp4"):
        mp4_topics.append(tname)
    print(tname, "partitions:", len(tmeta.partitions))
print("MP4 topics:", mp4_topics)

sess-wphp1s2v-metnfuq9 partitions: 1
gC_12_s5_2019-03-14T09-56-52-01-00_rgb_face-label.mp4 partitions: 1
sess-37p61fgw-mesoq67n partitions: 1
gC_14_s5_2019-03-12T09-18-58-01-00_rgb_face-label.mp4 partitions: 1
sess-rucqrv95-metgmqz9 partitions: 1
sess-5t1toq4k-mes66sgo partitions: 1
sess-4wnf2c2g-metmesyx partitions: 1
drowsiness_detection partitions: 1
sess-zrl0v70k-metn0gx0 partitions: 1
gC_15_s5_2019-03-12T11-03-23-01-00_rgb_face-label.mp4 partitions: 1
gC_11_s5_2019-03-12T09-08-15-01-00_rgb_face-label.mp4 partitions: 1
test-topic partitions: 1
zolgima-control partitions: 1
sess-igjk67gw-mep4nsua partitions: 1
sess-penuvo24-metgrljb partitions: 1
zolgima-test partitions: 1
dongan-test-topic partitions: 1
__consumer_offsets partitions: 50
gC_13_s5_2019-03-12T10-03-00-01-00_rgb_face-label.mp4 partitions: 1
MP4 topics: ['gC_12_s5_2019-03-14T09-56-52-01-00_rgb_face-label.mp4', 'gC_14_s5_2019-03-12T09-18-58-01-00_rgb_face-label.mp4', 'gC_15_s5_2019-03-12T11-03-23-01-00_rgb_face-label.mp4

In [7]:
frames_data = []
running = True

def handle_sigint(signum, frame):
    global running
    running = False

conf = {
        **kafka_conf,
        "group.id": "zolgima-test-consumer-group",
        "enable.auto.commit": False,  # 수동 커밋
        "auto.offset.reset": "earliest",  # 처음부터 읽고 싶을 때
        "session.timeout.ms": 45000,
}

c = Consumer(conf)
c.subscribe([mp4_topics[0]])

signal.signal(signal.SIGINT, handle_sigint)
print(f"👂 Subscribed to {mp4_topics[0]}. Press Ctrl+C to stop.")

try:
       while running:
              msg = c.poll(1.0)  # 1초 대기
              if msg is None:
                     continue
              if msg.error():
                # 파티션 EOF 등 무해한 에러 처리
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                raise KafkaException(msg.error())

            # 정상 메시지 처리
              try:
                     value = msg.value()
                     data = json.loads(value.decode("utf-8")) if value else None
                     key = msg.key().decode("utf-8") if msg.key() else None
                     print(f"[{msg.topic()} p{msg.partition()} @ {msg.offset()}] "
                     f"key={key} data={data}")
                     frames_data.append(data)
              except json.JSONDecodeError as e:
                     print(f"⚠ JSON decode error: {e}; raw={msg.value()!r}")
finally:
       print("🛑 Closing consumer...")
       c.close()

👂 Subscribed to gC_12_s5_2019-03-14T09-56-52-01-00_rgb_face-label.mp4. Press Ctrl+C to stop.
[gC_12_s5_2019-03-14T09-56-52-01-00_rgb_face-label.mp4 p0 @ 0] key=drowniness_indicator data={'frame': 1, 'time_sec': 0.033602150537634407, 'label_id': 0, 'label_name': 'eyes_state/open', 'EAR': 0.23353539258047856, 'MAR': 0.003968072015843257, 'yawn_rate_per_min': 0.0, 'blink_rate_per_min': 0.0, 'avg_blink_dur_sec': nan, 'longest_eye_closure_sec': nan, 'drowsiness_level': 0, 'drowsiness_label': 'alert'}
[gC_12_s5_2019-03-14T09-56-52-01-00_rgb_face-label.mp4 p0 @ 1] key=drowniness_indicator data={'frame': 2, 'time_sec': 0.06720430107526881, 'label_id': 0, 'label_name': 'eyes_state/open', 'EAR': 0.22623145485555213, 'MAR': 0.002633834758710877, 'yawn_rate_per_min': 0.0, 'blink_rate_per_min': 0.0, 'avg_blink_dur_sec': nan, 'longest_eye_closure_sec': nan, 'drowsiness_level': 0, 'drowsiness_label': 'alert'}
[gC_12_s5_2019-03-14T09-56-52-01-00_rgb_face-label.mp4 p0 @ 2] key=drowniness_indicator data

In [8]:
frame_df = pd.DataFrame(frames_data)
fps = 1 / frame_df.iloc[0]['time_sec']
window_size = 10.      # 초 단위 윈도우
hop_size = 1.         # 초 단위 홉

frame_window_size = int(round(fps * window_size, -1)) # 프레임 단위 윈도우(10단위 반올림)
frame_window_size

300

In [9]:
def make_segments(df, frame_col, label_col, time_col, insert_gaps=True):
    d = df.sort_values(by=frame_col).reset_index(drop=True).copy()
    d[frame_col] = d[frame_col].astype(int)
    d[time_col] = round(d[time_col].astype(float), 3)
    d[label_col] = d[label_col].astype(int)
    if d.empty: 
        return []

    segs = []
    start = int(d.loc[0, frame_col])
    start_time = float(d.loc[0, time_col])
    prev_frame = start
    prev_label = int(d.loc[0, label_col])

    for i in range(1, len(d)):
        f = int(d.loc[i, frame_col])
        t = float(d.loc[i, time_col])
        lab = int(d.loc[i, label_col])
        # 프레임 갭(누락 프레임) 처리
        if f != prev_frame + 1:
            segs.append({"start": start, "end": prev_frame + 1, "label": prev_label, "note": "프레임 누락"})
            if insert_gaps and f > prev_frame + 1:
                segs.append({"start": prev_frame + 1, "end": f, "label": -1, "note": "무시"})  # 전이/무시
            start, start_time, prev_frame, prev_label = f, t, f, lab
            continue
        # 라벨 변경 경계
        if lab != prev_label:
            segs.append({"start": start, "end": f, "label": prev_label, "note": f"시간 표현: {start_time} ~ {t}"})
            start, start_time, prev_label = f, t, lab
        prev_frame = f

    segs.append({"start": start, "end": prev_frame + 1, "label": prev_label, "note": f"시간 표현: {start_time} ~ {t}"})
    return segs

def convert_csv_to_segments_json(df, fps=30):
    df = df[["frame", "time_sec", "drowsiness_level", "drowsiness_label"]]

    segments = make_segments(df, "frame", "drowsiness_level", "time_sec", insert_gaps=True)
    num_frames = int(df["frame"].max()) + 1

    friendly = {0:"정상", 1:"의심", 2:"주의", 3:"위험", -1:"무시"}
    class_map = [{"id": int(i), "name": friendly.get(int(i))} for i in friendly]

    payload = {
        "segments": segments
    }
    return payload

In [10]:
def parse_label_json(json_segment):
    segs = []
    
    def norm_one(d):
        # 키 변형 수용
        if isinstance(d, dict):
            # 유형 1: 명시 키
            s = d.get("start_frame", d.get("frame_start", d.get("start")))
            e = d.get("end_frame", d.get("frame_end", d.get("end")))
            lvl = d.get("level", d.get("lvl", d.get("label", d.get("state"))))
            if s is not None and e is not None and lvl is not None:
                return int(s), int(e), int(lvl)
            # 유형 2: frames: [s, e]
            if "frames" in d and isinstance(d["frames"], (list, tuple)) and len(d["frames"]) >= 2:
                s, e = d["frames"][:2]
                lvl = int(d.get("level", d.get("label", -1)))
                return int(s), int(e), int(lvl)
        return None

    for item in json_segment["segments"]:
       r = norm_one(item)
       if r: segs.append(r)

    # 정렬 & 병합은 생략(중복은 뒤값 우선)
    segs.sort(key=lambda x: (x[0], x[1]))
    return segs

def build_frame_labels(max_frame: int, segments, default_level):
    """
    0..max_frame 범위에 대해 프레임별 level 배열 만들기.
    """
    labels = np.full((max_frame+1,), default_level, dtype=np.int32)
    for s, e, lvl in segments:
        s = max(0, s); e = min(max_frame, e)
        if e >= s:
            labels[s:e+1] = int(lvl)
    return labels

In [11]:
def make_windows_for_pair(df, json_segment, window_sec, hop_sec, fps=30):
    """
    하나의 (CSV, JSON) 쌍에서 (X, y) 윈도우 생성
    - X: (num_win, seq_len, num_feat)
    - y: (num_win,)  # 윈도우 대표 레벨 (다수결)
    """
    win = max(1, int(round(window_sec * fps)))
    hop = max(1, int(round(hop_sec * fps)))

    # 프레임별 라벨
    segs = parse_label_json(json_segment)
    max_frame = int(df["frame"].max())
    frame_labels = build_frame_labels(max_frame, segs, default_level=-1)

    # 특성 행렬
    feats = df[FEATURE_COLS].astype(float).copy()
    # NaN 보정(앞채움→뒤채움→0)
    feats = feats.ffill().bfill().fillna(0.0)
    feat_np = feats.to_numpy(dtype=np.float32)
    frames = df["frame"].to_numpy(dtype=np.int32)

    # 프레임 인덱스 → 라벨 매핑
    label_by_row = np.array([frame_labels[f] if f <= max_frame else -1 for f in frames], dtype=np.int32)

    X_list, y_list = [], []
    start = 0
    last_start = len(df) - win
    while start <= last_start:
        end = start + win
        seq = feat_np[start:end, :]  # (win, feat)
        seq_labels = label_by_row[start:end]  # (win,)

        # 윈도우 대표 라벨: 다수결(동점이면 마지막 프레임값)
        cnt = Counter(seq_labels.tolist())
        most = max(cnt.items(), key=lambda kv: (kv[1], kv[0]))[0]
        y = most

        X_list.append(seq)
        y_list.append(y)
        start += hop
    
    if last_start < 0:
        # 데이터가 윈도우 길이보다 짧은 경우
        seq = np.zeros((win, len(FEATURE_COLS)), dtype=np.float32)
        seq[:len(feat_np), :] = feat_np
        cnt = Counter(label_by_row.tolist())
        most = max(cnt.items(), key=lambda kv: (kv[1], kv[0]))[0]
        y = most
        X_list.append(seq)
        y_list.append(y)

    if not X_list:
        return np.empty((0, win, len(FEATURE_COLS)), dtype=np.float32), np.empty((0,), dtype=np.int32)

    X = np.stack(X_list, axis=0).astype(np.float32)
    y = np.array(y_list, dtype=np.int32)
    return X, y

In [12]:
def build_numpy_dataset(df, json_segment, window_sec=5.0, hop_sec=1.0, fps=30):
    Xs, ys = [], []
    Xi, yi = make_windows_for_pair(df, json_segment, window_sec, hop_sec, fps)
    if len(Xi) > 0:
       Xs.append(Xi); ys.append(yi)
    if not Xs:
        raise RuntimeError("윈도우가 생성되지 않았습니다. 입력 파일/라벨을 확인하세요.")

    max_len = max(X.shape[1] for X in Xs)
    Xs_padded = [
        np.pad(X, ((0,0),(0,max_len-X.shape[1]),(0,0)), mode="constant")
        if X.shape[1] < max_len else X
        for X in Xs
    ]
    
    X = np.concatenate(Xs_padded, axis=0)
    y = np.concatenate(ys, axis=0)
    return X, y

In [28]:
segments = []
for i in range(math.ceil(len(frame_df) / frame_window_size)):
       df = frame_df.iloc[i*frame_window_size:(i+1)*frame_window_size]
       segment = convert_csv_to_segments_json(df=df, fps=fps)
       segments.append((df, segment))

In [43]:
pair = make_windows_for_pair(frame_df.iloc[0*frame_window_size:(0+1)*frame_window_size], convert_csv_to_segments_json(frame_df.iloc[0*frame_window_size:(0+1)*frame_window_size], fps=fps), window_sec=5.0, hop_sec=1.0, fps=fps)

In [44]:
pair[0].shape, pair[1].shape

((6, 149, 6), (6,))

In [47]:
segments[0][0].shape

(300, 12)

In [32]:
inputs, outputs = [], []
for i in range(len(segments)):
       X, y = build_numpy_dataset(segments[i][0], segments[i][1], window_sec=window_size, hop_sec=hop_size, fps=fps)
       inputs.append(X)
       outputs.append(y)

In [34]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [21]:
def fit_standardizer(X: np.ndarray):
    """
    채널별 표준화 스케일러(평균/표준편차) 반환.
    """
    # X shape: (N, T, C)
    mean = X.reshape(-1, X.shape[-1]).mean(axis=0)
    std = X.reshape(-1, X.shape[-1]).std(axis=0)
    std = np.where(std < 1e-8, 1.0, std)
    return mean.astype(np.float32), std.astype(np.float32)

def apply_standardizer(X: np.ndarray, mean: np.ndarray, std: np.ndarray):
    return (X - mean) / std

In [35]:
input_scaleds = []
for i in range(len(inputs)):
       mean, std = fit_standardizer(inputs[i])
       X_scaled = apply_standardizer(inputs[i], mean, std)
       input_scaleds.append(X_scaled)

In [36]:
def to_tf_dataset(X: np.ndarray, batch=64, shuffle=True):
    # y를 인덱스로 변환(-1..3 → 0..4)
    ds = tf.data.Dataset.from_tensor_slices(X)
    if shuffle:
        ds = ds.shuffle(min(len(X), 10000), seed=RANDOM_SEED)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

In [37]:
tf_datasets = []
for i in range(len(input_scaleds)):
       ds = to_tf_dataset(input_scaleds[i], batch=64, shuffle=True)
       tf_datasets.append(ds)

In [38]:
loaded_model = tf.keras.models.load_model("best_lstm.keras")
loaded_model.predict(tf_datasets[0])

ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 300, 6), found shape=(1, 298, 6)

**↑** 25.08.27 model에 들어가야 되는 입력의 크기와 slicing을 진행해 만들어진 입력의 shape가 달라지는 문제 발생...   
-> slicing한 데이터들을 입력 데이터로 가공하는 과정에서 생긴 문제 같아 보이는데 정확한 원인을 조사중

# 코드 설명
## 1. kafka 서버에서 consumer를 통해 topic을 지정해서 해당 topic의 messege를 불러온다. (Messege는 frame 마다의 정보를 담아있는 json 포맷 데이터)
## 2. messege를 코드 내부에서 csv, json 변수화 시켜 모델 입력에 들어갈 변수로 가공한다.
## 3. 가공은 영상을 읽을 때 나눌 window 크기 단위를 고려해서 영상을 slicing한다.(ex. window_size를 10초라고 하면 영상 slicing은 300프레임 단위)
## 4. slicing된 영상들을 model에 입력으로 넣어 각 프레임 구간 별 예측값을 출력으로 내보낸다.
### 해당 코드의 동작을 실시간 영상에 맞춰 돌아갈 수 있게끔 수정하는 작업을 추가로 진행해야 됨(현재는 파일 베이스)